<a href="https://colab.research.google.com/github/Pavan0704/Machine-Learning-Pipelines/blob/main/ML_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer # for missing data
from sklearn.preprocessing import OneHotEncoder # because of categorical data
from sklearn.preprocessing import MinMaxScaler # for scaling
from sklearn.tree import DecisionTreeClassifier # for prediction
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, chi2 #for feature selection
from sklearn.pipeline import Pipeline, make_pipeline # Import the Pipeline class

In [38]:
df = pd.read_csv('Titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [39]:
df.drop(columns=['PassengerId', 'Name', 'Ticket','Cabin'], inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,34.5,0,0,7.8292,Q
1,1,3,female,47.0,1,0,7.0000,S
2,0,2,male,62.0,0,0,9.6875,Q
3,0,3,male,27.0,0,0,8.6625,S
4,1,3,female,22.0,1,1,12.2875,S


In [40]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=42)


In [41]:
y_train.head()

,Survived
336,0
31,0
84,0
287,0
317,0


In [42]:
df.isnull().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,86
SibSp,0
Parch,0
Fare,1
Embarked,0


In [43]:
si_age = SimpleImputer()
si_fare = SimpleImputer(strategy='most_frequent') #most frequnt = mode

X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_fare = si_fare.fit_transform(X_train[['Fare']])

X_test_age = si_age.transform(X_test[['Age']])
X_test_fare = si_fare.transform(X_test[['Fare']])

In [44]:
#one hot encoding
ohe_sex = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train[['Embarked']])

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test[['Embarked']])

X_train_embarked


array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [45]:
X_train_rem = X_train.drop(columns=['Sex', 'Age', 'Embarked'])
X_test_rem = X_test.drop(columns=['Sex', 'Age', 'Embarked'])

In [46]:
X_train_transformed = np.concatenate((X_train_rem, X_train_age, X_train_sex, X_train_embarked), axis=1)
X_test_transformed = np.concatenate((X_test_rem, X_test_age, X_test_sex, X_test_embarked), axis=1)

**Lets Plan**

In [47]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,2,male,32.0,0,0,13.0000,S
31,2,male,24.0,2,0,31.5000,S
84,2,male,NaN,0,0,10.7083,Q
287,1,male,24.0,1,0,82.2667,S
317,2,male,19.0,0,0,10.5000,S


In [48]:
y_train.sample(2)

,Survived
34,0
160,1


In [55]:
# imputation transformer
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]), # 2 is the index number of the column age
    ('impute_fare', SimpleImputer(strategy='most_frequent'), [5])
], remainder='passthrough')

In [57]:
# one hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_fare',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,5])
],remainder='passthrough')

In [58]:
# Scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [62]:
# Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [63]:
# train the model
trf5 = DecisionTreeClassifier()

**Create Pipeline**

In [89]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

**Pipeline Vs make_pipeline**

In [69]:
# Alternate Syntax
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [70]:
# train
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_fare',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [5])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_fare',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 5])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x78a0a01ef600>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [90]:
# Code here
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_fare',
                                  SimpleImputer(strategy='most_frequent'),
                                  [5])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_fare',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 5])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x78a0a01ef600>),
 'trf5': DecisionTreeClassifier()}

In [97]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array([7.75])

In [72]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')

In [73]:
# Predict
y_pred = pipe.predict(X_test)

In [74]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

**Cross Validation using Pipeline**

In [75]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5952380952380952

**GridSearch using Pipeline**

In [81]:
# gridsearchcv
params = {
    'decisiontreeclassifier__max_depth':[1,2,3,4,5,None]
}

In [82]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_fare',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [5])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_fare',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          5])])),
                                       ('columntransformer-3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('selectkbest',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x78a0a01ef600>)),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5,
                                                               None]},
             scoring='accuracy')

In [84]:
grid.best_score_

np.float64(0.6467209407507915)

In [85]:
grid.best_params_

{'decisiontreeclassifier__max_depth': 1}

**Exporting the Pipeline**

In [86]:
# export
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [98]:
pipe = pickle.load(open('pipe.pkl','rb'))

In [99]:
# Assume user input
test_input2 = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype=object).reshape(1,7)

In [100]:
pipe.predict(test_input2)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0])